In [1]:
!pip install interpret

 already satisfied: scikit-learn>=0.18.1 in /Users/beeyan/opt/anaconda3/lib/python3.8/site-packages (from interpret-core[dash,debug,decisiontree,ebm,lime,linear,notebook,plotly,required,sensitivity,shap,skoperules,treeinterpreter]>=0.2.4->interpret) (0.23.2)
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 3.5 MB 12.5 MB/s 
     |████████████████████████████████| 188 kB 11.8 MB/s 
     |████████████████████████████████| 1.0 MB 12.9 MB/s 
     |████████████████████████████████| 1.8 MB 11.7 MB/s 
  Using cached dash_cytoscape-0.2.0-py3-none-any.whl
     |████████████████████████████████| 81 kB 5.2 MB/s 
  Using cached Flask-1.1.2-py2.py3-none-any.whl (94 kB)
     |████████████████████████████████| 1.8 MB 7.0 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
  Using cached itsdangerous-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached lime-0.2.0.1-py3-none-any.whl
     |████████████████████████████████| 13.2 MB 12.1 MB/s 
  Using 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]
# df = df.sample(frac=0.1, random_state=1)
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [3]:
from interpret import show
from interpret.data import ClassHistogram

# Histgram objectを作成して, showに渡すことで, 描画ができる
hist = ClassHistogram().explain_data(X_train, y_train, name='Train Data')
show(hist)

<!-- http://127.0.0.1:7001/140656000709680/ -->

In [4]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier

# EBMインスタンスを作成して, explain_global instanceを作成してshow関数に渡す
ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)

# Global Explanations: 
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7001/140656068456944/ -->

In [5]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7001/140656051637408/ -->

In [6]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<!-- http://127.0.0.1:7001/140656051420128/ -->

In [7]:
from interpret.glassbox import LogisticRegression, ClassificationTree

# We have to transform categorical variables to use Logistic Regression and Decision Tree
X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed, feature_names=feature_names, penalty='l1', solver='liblinear')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

In [8]:
lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)
show(ebm_perf)

<!-- http://127.0.0.1:7001/140656051627344/ -->

<!-- http://127.0.0.1:7001/140656069531920/ -->

<!-- http://127.0.0.1:7001/140656051420128/ -->

In [9]:
lr_global = lr.explain_global(name='Logistic Regression')
tree_global = tree.explain_global(name='Classification Tree')

show(lr_global)
show(tree_global)
show(ebm_global)

<!-- http://127.0.0.1:7001/140656051705312/ -->

<!-- http://127.0.0.1:7001/140656089816368/ -->

<!-- http://127.0.0.1:7001/140656068456944/ -->

In [10]:
show([hist, lr_global, lr_perf, tree_global, tree_perf, ebm_global, ebm_perf], share_tables=True)

<!-- http://127.0.0.1:7001/140656083864896/ -->
 Open in new window